In [2]:
import pandas as pd
import datetime
import time
import pickle
import numpy as np
from sqlalchemy import create_engine, inspect


In [2]:
database_path = "restaurants.sqlite"
conn_string = f"sqlite:///{database_path}"

# Create an engine that can talk to the database
engine = create_engine(conn_string)


In [3]:
 # get all tables
inspector_gadget = inspect(engine)
tables = inspector_gadget.get_table_names()
for table in tables:
    print(table)
    
    # get all columns in table
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column)
    print()

AlcoholType
{'name': 'index', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'id', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'description', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}

BYOBCorkageType
{'name': 'index', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'id', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'description', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}

BestNightsType
{'name': 'index', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'id', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'description', 'type': TEXT(), 'nul

In [68]:
city = '"Tempe"'
state = '"AZ"'
mexican_restaurant = False
query = f"""
        SELECT
              name,
              address,
              city,
              state,
              postal_code,
              review_count,
              stars
        FROM
            Restaurants
        WHERE
            city IN ({city})
            AND state IN ({state})
            And is_mexican_restaurant = {mexican_restaurant};

           
            """     
df = pd.read_sql(query, con=engine)
df.head()

,name,address,city,state,postal_code,review_count,stars
0,"""Wong's Place""","""1825 E Baseline Rd.""",Tempe,AZ,85283,149,4.0
1,"""Subway""","""130 E University Dr, Ste C""",Tempe,AZ,85281,5,3.5
2,"""Riazzi's Italian Garden""","""2700 S Mill Ave""",Tempe,AZ,85282,141,3.5
3,"""WisePies Pizza & Salad""","""521 S College Ave, Ste 110""",Tempe,AZ,85281,14,4.0
4,"""Silver Mine Subs""","""655 W Warner Rd""",Tempe,AZ,85284,14,3.5


In [84]:
dietary_restictions = '"na"'
meal_type = '"na"'
mexican_restaurant = False
restraunt_type = '"na"'
query2 = f"""
        SELECT
              r.name,
              r.address,
              r.city,
              r.state,
              r.postal_code,
              r.review_count,
              r.stars
        FROM
            Restaurants as r
        INNER JOIN 
            RestaurantAttributes as ra
            ON r.business_id = ra.business_id        
        WHERE
            city IN ({city})
            AND state IN ({state})
            AND ra.DietaryRestrictionsType = {dietary_restictions}
            AND ra.GoodForMealType = {meal_type}
            And is_mexican_restaurant = {mexican_restaurant}
            And RestaurantsType = {restraunt_type};
           
            """     
attribute_df = pd.read_sql(query2, con=engine)
attribute_df.head()

,name,address,city,state,postal_code,review_count,stars
0,"""Wong's Place""","""1825 E Baseline Rd.""",Tempe,AZ,85283,149,4.0
1,"""Subway""","""130 E University Dr, Ste C""",Tempe,AZ,85281,5,3.5
2,"""Riazzi's Italian Garden""","""2700 S Mill Ave""",Tempe,AZ,85282,141,3.5
3,"""WisePies Pizza & Salad""","""521 S College Ave, Ste 110""",Tempe,AZ,85281,14,4.0
4,"""Silver Mine Subs""","""655 W Warner Rd""",Tempe,AZ,85284,14,3.5


In [65]:
attribute_df.DietaryRestrictionsType.value_counts()

na                               13828
DietaryRestrictionsvegetarian       14
DietaryRestrictionsvegan             6
Name: DietaryRestrictionsType, dtype: int64

In [66]:
attribute_df.GoodForMealType.value_counts()

na                      13599
GoodForMealdinner         134
GoodForMeallunch           57
GoodForMealbrunch          25
GoodForMealbreakfast       24
GoodForMealdessert          8
GoodForMeallatenight        1
Name: GoodForMealType, dtype: int64

In [72]:
attribute_df.RestaurantsType.value_counts()

na                          11558
RestaurantsGoodForGroups      404
RestaurantsTakeOut            273
RestaurantsTableService       230
RestaurantsDelivery            69
RestaurantsReservations        29
Name: RestaurantsType, dtype: int64

In [85]:
query3 = f"""
        SELECT
              r.name,
              rev.text,
              rev.likes

        FROM
            Reviews as rev
        INNER JOIN 
            Restaurants as r
            ON rev.business_id = r.business_id
                    INNER JOIN 
                        RestaurantAttributes as ra
                                    ON r.business_id = ra.business_id
        WHERE
            city IN ({city})
            AND state IN ({state})
            AND ra.DietaryRestrictionsType = {dietary_restictions}
            AND ra.GoodForMealType = {meal_type}
            And is_mexican_restaurant = {mexican_restaurant}
            And RestaurantsType = {restraunt_type};
            """     
tips_df = pd.read_sql(query3, con=engine)
tips_df.head()

,name,text,likes
0,"""Wong's Place""",6 orders salt-n-pepper pork!,0
1,"""Wong's Place""","Garlic fried shrimp, with MSG laced broccoli!",0
2,"""Wong's Place""",5 order SnPP!,0
3,"""Wong's Place""",Garlic fried shrimp... Order it just for the ...,1
4,"""Wong's Place""",Stuffing our face with authentic chinese food ...,0
